# Data preprocessing

Data cleaning and aggregation part. The main dataset is Movielens' 25 million movie review dataset. IMDb's official datasets is used to enrich the main dataset with potentially relevant information such as director and writer.

In [82]:
# Libraries imports and function declarations
import pandas as pd
import numpy as np
import json

# 1. Movies

## Movielens dataset

Data cleaning of the films dataset.

In [83]:
def convert_genres_list(x):
    if not x:
        return np.NaN
    # If we rearch this point, it means we've got a string
    # with at least 1 genre.
    arr = x.lower().split("|")

    # For simplicity we'll only return the first 3 genres.
    # If N < 3, len(arr) = N.
    return arr[:3]

df_main = pd.read_csv(
    "datasets/ml-25m/movies.csv", sep=",",
    converters= {
        'genres': convert_genres_list
    })

In [84]:
df_main.head(2)

,movieId,title,genres
0,1,Toy Story (1995),"[adventure, animation, children]"
1,2,Jumanji (1995),"[adventure, children, fantasy]"


In [85]:
df_main.shape

(62423, 3)

In [86]:
df_main.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [87]:
df_main['title'].duplicated().all()

False

In [88]:
def process_year(movie):
    # Format: movie_name (year)
    # We'll try to get the highest index for the parenthesis.
    # If any of them are not found or the cast has an invalid input return -1 as year.
    try:
        start = movie.rindex('(')
        end = movie.rindex(')')
        
        year = int(movie[start+1:end])
        return year
    except:
        return -1    
    
df_main['year'] = df_main['title'].apply(lambda x: process_year(x))

In [89]:
-1 in df_main['year'] # All years have been successfully converted

False

In [90]:
# Delete year from name
def process_name(movie):
    # Format: movie_name (year)
    # We'll try to get the highest index for the parenthesis.
    try:
        start = movie.rindex('(')
        new_movie = movie[:start].strip()
        return new_movie
    except:
        return movie

df_main['title'] = df_main['title'].apply(lambda x: process_name(x))

In [91]:
df_main.head(5)

,movieId,title,genres,year
0,1,Toy Story,"[adventure, animation, children]",1995
1,2,Jumanji,"[adventure, children, fantasy]",1995
2,3,Grumpier Old Men,"[comedy, romance]",1995
3,4,Waiting to Exhale,"[comedy, drama, romance]",1995
4,5,Father of the Bride Part II,[comedy],1995


Once we have this dataset cleaned, we need to aggregate information from the IMDb dataset.

Informatio such as titleType, director, writer and main actors will be added.

## IMDb dataset

In [92]:
df_im = pd.read_csv(
    "datasets/IMDb/title_basics.tsv", sep="\t",
    usecols= ['tconst', 'titleType', 'primaryTitle']
    )

In [93]:
df_im.shape

(8699991, 3)

In [94]:
df_im.isnull().sum()

tconst          0
titleType       0
primaryTitle    8
dtype: int64

In [95]:
df_im['titleType'].unique()

array(['short', 'movie', 'tvEpisode', 'tvSeries', 'tvShort', 'tvMovie',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame', 'tvPilot'],
      dtype=object)

In [96]:
# We must drop all columns whose type is not short, movie, tvShort or tvMovie.
# (we're unsure whether Movielense's dataset includes shorts, we'll keep them just in case).
df_im = df_im.loc[df_im['titleType'].isin([
        'short', 'movie', 'tvShort', 'tvMovie'
    ])]

In [97]:
df_im.shape # ~7 million rows dropped.

(1603640, 3)

In [98]:
df_im.isnull().sum()

tconst          0
titleType       0
primaryTitle    0
dtype: int64

### Adding directors and writers

In [99]:
def convert_list(x):
    if not x:
        return np.NaN
    if x == '\\N':
        return np.NaN
    return x.split(',')[0]

df_crew = pd.read_csv(
    "datasets/IMDb/title_crew.tsv", sep="\t",
    converters= {
        'directors': convert_list,
        'writers': convert_list
    })

In [100]:
df_crew.head(3)

,tconst,directors,writers
0,tt0000001,nm0005690,NaN
1,tt0000002,nm0721526,NaN
2,tt0000003,nm0721526,NaN


In [101]:
df_im = pd.merge(df_im, df_crew, how='left', on=['tconst'])

In [102]:
df_im.head(3)

,tconst,titleType,primaryTitle,directors,writers
0,tt0000001,short,Carmencita,nm0005690,NaN
1,tt0000002,short,Le clown et ses chiens,nm0721526,NaN
2,tt0000003,short,Pauvre Pierrot,nm0721526,NaN


In [103]:
df_im.shape

(1603640, 5)

In [104]:
df_name = pd.read_csv(
    "datasets/IMDb/name_basics.tsv", sep="\t",
    usecols=[
        'nconst',
        'primaryName'
    ])

In [105]:
df_name.head(3)

,nconst,primaryName
0,nm0000001,Fred Astaire
1,nm0000002,Lauren Bacall
2,nm0000003,Brigitte Bardot


In [106]:
df_im = pd.merge(df_im, df_name,
    how='left',
    left_on=['directors'], right_on=['nconst']
    )

# Drop extra column, rename director's column name
df_im = df_im.drop(columns='nconst')
df_im = df_im.rename(columns={'primaryName':'directorName'})

In [107]:
df_im = pd.merge(df_im, df_name,
    how='left',
    left_on=['writers'], right_on=['nconst']
    )

df_im = df_im.drop(columns='nconst')
df_im = df_im.rename(columns={'primaryName':'writerName'})


In [108]:
df_im = df_im.drop(columns=['directors','writers'])
df_im = df_im.rename(columns={
    'directorName':'director',
    'writerName':'writer'
    })


In [109]:
df_im.head(10)

,tconst,titleType,primaryTitle,director,writer
0,tt0000001,short,Carmencita,William K.L. Dickson,NaN
1,tt0000002,short,Le clown et ses chiens,Émile Reynaud,NaN
2,tt0000003,short,Pauvre Pierrot,Émile Reynaud,NaN
3,tt0000004,short,Un bon bock,Émile Reynaud,NaN
4,tt0000005,short,Blacksmith Scene,William K.L. Dickson,NaN
5,tt0000006,short,Chinese Opium Den,William K.L. Dickson,NaN
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,William Heise,NaN
7,tt0000008,short,Edison Kinetoscopic Record of a Sneeze,William K.L. Dickson,NaN
8,tt0000009,short,Miss Jerry,Alexander Black,Alexander Black
9,tt0000010,short,Leaving the Factory,Louis Lumière,NaN


## Merging the two datasets

In [110]:
df_main.shape[0]

62423

In [111]:
len(set(df_main['title']).intersection(set(df_im['primaryTitle'])))

40916

IMDb's dataset contains information for 40916 of the 62423 movies from the Movielense dataset, or around 65% of the movies.
Since the dataset will be reduced anyway for memory reasons we can drop the movies with no extra information.

In [112]:
df_main = df_main.merge(df_im.drop_duplicates(subset=['primaryTitle']),
    left_on='title', right_on='primaryTitle',
    how='left')

In [113]:
df_main.shape

(62423, 9)

In [114]:
df_main.isnull().sum()

movieId             0
title               0
genres              0
year                0
tconst          17842
titleType       17842
primaryTitle    17842
director        18455
writer          21563
dtype: int64

In [115]:
# We can delete the rows there the tconst is NaN.
# df_main = df_main[df_main['tconst'].notna()]
df_main = df_main.dropna() # We drop all rows that have any NaN

In [116]:
df_main.columns

Index(['movieId', 'title', 'genres', 'year', 'tconst', 'titleType',
       'primaryTitle', 'director', 'writer'],
      dtype='object')

In [117]:
# Now, delete the useless extra columns and write the dataset to file.
df_main = df_main.drop(['tconst', 'primaryTitle'], axis=1)

In [118]:
df_main.head(3)

,movieId,title,genres,year,titleType,director,writer
0,1,Toy Story,"[adventure, animation, children]",1995,movie,John Lasseter,John Lasseter
1,2,Jumanji,"[adventure, children, fantasy]",1995,movie,Joe Johnston,Jonathan Hensleigh
2,3,Grumpier Old Men,"[comedy, romance]",1995,movie,Howard Deutch,Mark Steven Johnson


In [119]:
# Expand list of genres into new columns.

# 1st, expand genres into its own dataframe
genres = df_main['genres'].apply(pd.Series)

# Rename columns
genres = genres.rename(columns = lambda x : 'genre_' + str(x))

# view the tags dataframe
genres

,genre_0,genre_1,genre_2
0,adventure,animation,children
1,adventure,children,fantasy
2,comedy,romance,NaN
3,comedy,drama,romance
4,comedy,NaN,NaN
...,...,...,...
62416,comedy,drama,NaN
62419,documentary,NaN,NaN
62420,comedy,drama,NaN
62421,(no genres listed),NaN,NaN


In [120]:
df_main = pd.concat([df_main[:], genres[:]], axis=1)

# df_main.drop('genres', axis=1, inplace=True) # Having the genres column will be usefull for later on.


In [121]:
df_json = df_main.to_json(orient="records")
parsed = json.loads(df_json)

with open("datasets/final/movies_final.json", "w", encoding="utf-8") as f:
    json.dump(parsed, f, indent=4)

# 2. Ratings

For this dataset, the main idea is to delete reviews from films that aren't present in the previous dataset. 

In [122]:
df_rt = pd.read_csv(
    "datasets/ml-25m/ratings.csv",
    usecols=[
        'userId',
        'movieId',
        'rating'
    ])

In [123]:
df_rt.head(5)

,userId,movieId,rating
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5


In [124]:
df_rt.shape[0] # 25.000.095 ratings

25000095

In [125]:
# Added to skip reading datasets when executing all cells.
try:
    df_main
except NameError:
    var_exists = False
else:
    var_exists = True


if not var_exists:
    df_films = pd.read_json("datasets/final/movies_final.json", orient="records")
else:
    df_films = df_main

In [126]:
df_films.head(5)

,movieId,title,genres,year,titleType,director,writer,genre_0,genre_1,genre_2
0,1,Toy Story,"[adventure, animation, children]",1995,movie,John Lasseter,John Lasseter,adventure,animation,children
1,2,Jumanji,"[adventure, children, fantasy]",1995,movie,Joe Johnston,Jonathan Hensleigh,adventure,children,fantasy
2,3,Grumpier Old Men,"[comedy, romance]",1995,movie,Howard Deutch,Mark Steven Johnson,comedy,romance,NaN
3,4,Waiting to Exhale,"[comedy, drama, romance]",1995,movie,Forest Whitaker,Terry McMillan,comedy,drama,romance
4,5,Father of the Bride Part II,[comedy],1995,movie,Charles Shyer,Albert Hackett,comedy,NaN,NaN


In [127]:
df_films.shape[0]

40609

We must delete those reviews of films which don't exist anymore.

In [128]:
df_main = df_rt[df_rt['movieId'].isin(df_films['movieId'])]

In [129]:
df_main.shape[0] # 17.165.849

17165849

In [130]:
# Due to computational constraints, the number of reviews will be further reduced to 25%.
df_main = df_main.sample(frac=0.25)

In [131]:
df_main.shape[0]

4291462

In [132]:
df_json = df_main.to_json(orient="records")
parsed = json.loads(df_json)

with open("datasets/final/ratings_final.json", "w", encoding="utf-8") as f:
    json.dump(parsed, f, indent=4)

# 3. Average rating and std by film.

In [2]:
import pandas as pd
import numpy as np

In [3]:
df_films = pd.read_json("datasets/final/movies_final.json", orient="records")
df_rt = pd.read_json("datasets/final/ratings_final.json", orient="records")

In [4]:
# Create columns 'mean' and 'std'
df_films['mean'] = np.NaN
df_films['std'] = np.NaN

In [5]:
df_films.shape[0]

40609

In [6]:
df_rt.shape[0]

4291462

In [7]:
df_rt[df_rt['movieId'] == 296]['rating'].mean()

4.203605648008069

In [10]:
# Define functions to calculate mean and std.
def calculate_mean(x):
    return df_rt[df_rt['movieId'] == x]['rating'].mean()

def calculate_std(x):
    return df_rt[df_rt['movieId'] == x]['rating'].std()

In [11]:
df_films['mean'] = df_films['movieId'].apply(calculate_mean)

In [12]:
df_films['std'] = df_films['movieId'].apply(calculate_std)

In [35]:
df_films[['mean','std']].isnull().sum()

mean    14045
std     22337
dtype: int64

In [42]:
# We will drop the films with no reviews.
df_films = df_films[df_films['mean'].notna()]

In [43]:
df_films = df_films[df_films['std'].notna()]

In [44]:
df_films[['mean','std']].isnull().sum()

mean    0
std     0
dtype: int64

In [46]:
df_films.shape[0]

18272

In [48]:
df_json = df_films.to_json(orient="records")
parsed = json.loads(df_json)

with open("datasets/final/movies_final.json", "w", encoding="utf-8") as f:
    json.dump(parsed, f, indent=4)

In [49]:
# We have to update the reviews dataset.
df_rt = pd.read_json("datasets/final/ratings_final.json", orient="records")

df_rt = df_rt[df_rt['movieId'].isin(df_films['movieId'])]

df_json = df_rt.to_json(orient="records")
parsed = json.loads(df_json)

with open("datasets/final/ratings_reduced_50.json", "w", encoding="utf-8") as f:
    json.dump(parsed, f, indent=4)

In [55]:
df_rt.shape[0]

4283170

# 4. Reduced datasets

## 50%

In [51]:
try:
    df_films
except NameError:
    var_exists = False
else:
    var_exists = True


if not var_exists:
    df_films = pd.read_json("datasets/final/movies_final.json", orient="records")

# 1st, reduce the films dataset.
#df_films = pd.read_json("datasets/final/movies_final.json", orient="records")

df_films.shape[0]

18272

In [52]:
df_films = df_films.sample(frac=0.50)

df_films.shape[0]

9136

In [53]:
df_json = df_films.to_json(orient="records")
parsed = json.loads(df_json)

with open("datasets/final/movies_reduced_50.json", "w", encoding="utf-8") as f:
    json.dump(parsed, f, indent=4)

In [54]:
try:
    df_rt
except NameError:
    var_exists = False
else:
    var_exists = True

if not var_exists:
    df_rt = pd.read_json("datasets/final/ratings_final.json", orient="records")


# Now ratings dataset
#df_rt = pd.read_json("datasets/final/ratings_final.json", orient="records")

df_rt.shape[0]

4283170

In [56]:
df_rt = df_rt[df_rt['movieId'].isin(df_films['movieId'])]

df_rt.shape[0]

2307873

In [57]:
df_json = df_rt.to_json(orient="records")
parsed = json.loads(df_json)

with open("datasets/final/ratings_reduced_50.json", "w", encoding="utf-8") as f:
    json.dump(parsed, f, indent=4)

## 25%

In [58]:
import pandas as pd
import json

In [59]:
# 1st, reduce the films dataset.
df_films = pd.read_json("datasets/final/movies_final.json", orient="records")

df_films = df_films.sample(frac=0.25)


df_json = df_films.to_json(orient="records")
parsed = json.loads(df_json)

with open("datasets/final/movies_reduced_25.json", "w", encoding="utf-8") as f:
    json.dump(parsed, f, indent=4)

In [60]:
# Now ratings dataset
df_rt = pd.read_json("datasets/final/ratings_final.json", orient="records")

df_rt = df_rt[df_rt['movieId'].isin(df_films['movieId'])]

df_json = df_rt.to_json(orient="records")

# Even with the reduced dataset, there are
# ~3 million reviews. Still way too many for
# our computing power. That's why it is reduced
# to ~300K.
# df_films = df_films.sample(frac=0.10)

parsed = json.loads(df_json)

with open("datasets/final/ratings_reduced_25.json", "w", encoding="utf-8") as f:
    json.dump(parsed, f, indent=4)

In [61]:
df_rt.shape[0]

1060005